In [5]:
from collections import deque
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import codecs
import re
from webdriver_manager.chrome import ChromeDriverManager

In [6]:
from selenium.webdriver.chrome.options import Options

# Set up Chrome options
chrome_options = Options()
service = Service(executable_path='C:\Program Files (x86)\chromedriver.exe')
chrome_options.binary_location = 'C:\Program Files\Google\Chrome\Application\chrome.exe'  # Specify the path to the Chrome browser executable if needed
#chrome_options.add_argument('--headless')  # Run Chrome in headless mode

# Set up Chrome WebDriver with the local ChromeDriver executable
driver = webdriver.Chrome(options=chrome_options, service=service)

driver.get("https://www.ip-label.fr/")
anchor_elements = driver.find_elements(By.CSS_SELECTOR, "input[type='search']")

In [15]:
for element in anchor_elements:
    
    tag_name = element.tag_name
    classs = element.get_attribute("class")
    print(classs)
    id = element.get_attribute("id")
    print(id)
    name = element.get_attribute("name")
    print(name)
    
    aria_label = element.get_attribute("aria-label")
    print(aria_label)
    
    xpath = f"//{tag_name}[@class='{classs}' and @id='{id}' and @name='{name}' and @aria-label='{aria_label}']"
    driver.find_element(By.XPATH, xpath)
    print(driver)


field
ocean-search-form-1
s
Insert search query
<selenium.webdriver.chrome.webdriver.WebDriver (session="0c483f30addeca0c87d678d765638707")>
field
ocean-mobile-search-2
s
Insert search query
<selenium.webdriver.chrome.webdriver.WebDriver (session="0c483f30addeca0c87d678d765638707")>


In [4]:


driver.get("https://www.ip-label.fr/")
anchor_elements = driver.find_elements(By.CSS_SELECTOR, "input[type='search']")
elements_set = set()
def get_element_xpath(elements_set,element):
    # Get the tag name of the element
    tag_name = element.tag_name

    # Get the attributes of the element
    attributes = element.get_attribute("outerHTML")
    
    print(attributes)
    
    data = element.get_attribute('data-value')

    # Construct the XPath manually using the tag name and attributes
    xpath = f"//{tag_name}[{attributes}]"
    length = len(elements_set)
    elements_set.add(xpath)

    # Print the generated XPath
    if length == len(elements_set) :
        print("Generated XPath:", xpath)

for element in anchor_elements:
    get_element_xpath(elements_set,element)
    
print(len(elements_set))
print(len(anchor_elements))




    
    

<input aria-label="Insert search query" type="search" id="ocean-search-form-1" class="field" autocomplete="off" placeholder="Rechercher" name="s">
<input aria-label="Insert search query" value="" class="field" id="ocean-mobile-search-2" type="search" name="s" autocomplete="off" placeholder="Rechercher">
2
2


In [ ]:
from selenium.webdriver.support.ui import WebDriverWait

driver.get("https://www.ip-label.fr/")
driver.find_elements(By.CSS_SELECTOR, "input[type='search']")


for i in range(5):
    new_tab_handle = driver.window_handles[0]
    driver.switch_to.window(new_tab_handle)
    url=anchor_elements[i].get_attribute('href')
    # Execute JavaScript to open a new tab.
    script = f"window.open('{url}', '_blank');"
    driver.execute_script(script)

    # Switch to the new tab.
    new_tab_handle = driver.window_handles[i+1]
    driver.switch_to.window(new_tab_handle)
    wait = WebDriverWait(driver, 2)

    print("The new tab is now open.")

In [ ]:
def get_page_elements(driver,next=False,element=None):
    # Extract all URLs and navigation data
    anchor_elements = driver.find_elements(By.TAG_NAME, 'a')
    all_elements = [{"type":"button","element":element,"next":[]} for element in anchor_elements]
    input_elements = driver.find_elements(By.CSS_SELECTOR, "input[type='search']")
    all_elements.extend([{"type":"input","element":element,"next":[]} for element in input_elements])
    if next:
            element["next"].extend(anchor_elements)
            element["next"].extend(input_elements)
    else :
        return {"page" : driver.current_url, "elements": all_elements}
    
def add_element(driver,all_elements=[]):
    all_elements.append(get_page_elements(driver))
    for all_element in all_elements:
        if all_element["page"] == driver.current_url:
            current_elements = all_element["elements"]
            break
        else:
            current_elements = []
    for element in current_elements:
        print(element)
        if element["type"] == "button":
            current_url = driver.current_url
            try:
                print(current_url)
                element["element"].click()
                new_url=driver.current_url
                if new_url != current_url:
                    test=False
                    for element_exist in all_elements:
                        if element_exist["page"] == new_url:
                            test=True
                    if not test:
                        get_page_elements(driver,True,element=element)
                    else:
                        for new_url_elements in all_elements:
                            next_elements = new_url_elements["elements"] if new_url_elements["page"] == new_url else None
                            for next_element in next_elements:
                                element["next"].append(next_element["element"])
            except ElementNotInteractableException:
                print("Non clickable")
                
        else:
            text_to_write = "Hello"
            element.send_keys(text_to_write)
    return all_elements
    
    

In [ ]:
def generate_scenarios(url):
    driver.get(url)
    list = add_element(driver)
    print(list)
    
    


In [ ]:
generate_scenarios("https://computer-database.gatling.io/computers?p=6")